# Generate Test Suite Notebooks

This bit of Python code will generate all the notebooks in the `TestSuite` directory, so if you execute this, the contents of TestSuite will be replaced.

This code takes all the test files from `/sysml/src/examples/`, i.e. `Import Tests` and `Simple Tests`, and for each it will create a separate notebook in the TestSuite directory.
 
**NOTE**: This does not need to be run since all the notebooks have already been generated and executed. This is should only be run locally to update the notebooks. Having said that, there is nothing preventing you from running this, it will simply generate the same output.


In [1]:
import json, re, os, subprocess
from glob import glob

def empty_nb():
    return {
     "cells": [
        {
       "cell_type": "markdown",
       "metadata": {},
       "source": [
        "# SysML v2 Test Suite\n",
        "\n",
           "This can notebook can be cleared using menu `Edit --> Clear All Outputs` and then all cells can be\n",
           "run using menu `Run --> Run All Cells`.\n",
        "\n",
           "**WARNING**: This file is generated, all modifications here will be ignored.\n"
       ]
       }
     ],
    
     "metadata": {
       "kernelspec": {
        "display_name": "SysML",
        "language": "sysml",
        "name": "sysml"
       },
       "language_info": {
        "codemirror_mode": "sysml",
        "file_extension": ".sysml",
        "mimetype": "text/x-sysml",
        "name": "SysML",
        "pygments_lexer": "java",
        "version": "1.0.0"
       }
     },
     "nbformat": 4,
     "nbformat_minor": 5
    }

package_name_re = re.compile("(package|action def) ([^\s]+)", re.I)

subprocess.call("rm -fr TestSuite ; mkdir TestSuite", shell=True)

viz_styles = ["COMPTREE", "POLYLINE", "LR", "STDCOLOR", "PLANTUML", "DEFAULT", "TB", "ORTHOLINE" ]
viz_views  = ["Default", "Tree", "State", "Interconnection", "Action", "Sequence",  "MIXED" ]


for flname in glob("../../sysml/src/examples/*Tests/*.sysml"):
    lnes   = open(flname, "r").readlines()
    nbname = os.path.basename(flname).replace("sysml", "ipynb")
    
    nbcnt = empty_nb()
    nbcnt["cells"].append({
        "cell_type": "markdown",
        "metadata": {},
        "source": "## File: {}".format(flname)
    })
    nbcnt["cells"].append({
        "cell_type": "code",
        "execution_count": 0,
        "metadata": {},
        "outputs": [],
        "source": lnes
    })
    
    md = package_name_re.match(lnes[0])
    if md != None:
        package_name = md[2]
        for style in viz_styles:
            for view in viz_views:
                nbcnt["cells"].append({
                    "cell_type": "code",
                    "execution_count": 0,
                    "metadata": {},
                    "outputs": [],
                    "source": "%viz --style=\"{}\" --view=\"{}\" \"{}\"".format( style, view, package_name)
                })
        
    f = open("TestSuite/{}".format(nbname),"w+")
    f.write(json.dumps(nbcnt))
    f.close()
    
    # to display data, i.e. svgs, need to trust the notebook.
    subprocess.call("jupyter trust TestSuite/{}".format(nbname), shell=True)

## Generate example files from the examples

This takes the specific examples from the examples directory and creates notebooks from this them. This is a little more tricky since the example usually have multiple files that need to be included in a single notebook. Also ordering is important so the files are listed in a specific order for the notebook to run.

In [2]:
import json, re, os, subprocess
from glob import glob

subprocess.call("rm -fr TestSuite/Examples ; mkdir -p TestSuite/Examples", shell=True)
basedir = "../../sysml/src/examples/"

examples = {
    "Analysis Examples": [
        "AnalysisAnnotation.sysml", 
        "Dynamics.sysml", 
        "Turbojet Stage Analysis.sysml", 
        "Vehicle Analysis Demo.sysml"
    ],
    "Arrowhead Framework Example": [
        "AHFProfileLib.sysml",
        "AHFCoreLib.sysml",  
        "AHFNorwayTopics.sysml"
    ],
    "Camera Example": [
        "PictureTaking.sysml",
        "Camera.sysml"
    ],
    "Comment Examples": [
        "Comments.sysml"
    ],
    "Mass Roll-up Example": [
        "MassConstraintExample.sysml",
        "MassRollup.sysml",
        "Vehicles.sysml"
    ],
    "Packet Example": [
        "Packets.sysml",
        "PacketUsage.sysml"
    ],
    "Requirements Examples": [
        "HSUVRequirements.sysml"
    ],
    "Room Model": [
        "RoomModel.sysml"
    ],
    "Variability Examples": [
        "VehicleVariabilityModel.sysml"
    ],
    "Vehicle Example": [
        "VehicleDefinitions.sysml",
        "VehicleUsages.sysml",
        "VehicleIndividuals.sysml",
        "VehicleModel_1_Simplified_05_21_21.sysml",
        "SysML v2 Spec Annex B VehicleModel.sysml"
    ],
}

for exmpldir in examples:
    nbname = exmpldir + ".ipynb"
    nbcnt = empty_nb()

    nbcnt["cells"].append({
        "cell_type": "markdown",
        "metadata": {},
        "source": "## Base Example Directory: {}".format(basedir + exmpldir)
    })

    for flname in examples[exmpldir]:
        lnes = open(basedir + exmpldir + "/" + flname, "r").readlines()
        nbcnt["cells"].append({
            "cell_type": "markdown",
            "metadata": {},
            "source": "## File: {}".format(flname)
        })
        nbcnt["cells"].append({
            "cell_type": "code",
            "execution_count": 0,
            "metadata": {},
            "outputs": [],
            "source": lnes
        })

    f = open("TestSuite/Examples/{}".format(nbname),"w+")
    f.write(json.dumps(nbcnt))
    f.close()
    
    # to display data, i.e. svgs, need to trust the notebook.
    subprocess.call("jupyter trust 'TestSuite/Examples/{}'".format(nbname), shell=True)


## Execute Test Suite

Having generated the test suite from the test SysML files, run through all the notebooks and execute them all. Replace the notebooks with the output.

In [3]:
import glob, nbformat, subprocess

from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert.preprocessors.execute import CellExecutionError

ExecutePreprocessor.allow_errors = True

for flname in (["KernelMagic.ipynb"] + glob.glob("SysML*.ipynb") + 
               glob.glob("TestSuite/*.ipynb") + glob.glob("TestSuite/Examples/*.ipynb")):
    with open(flname) as f:
        nb    = nbformat.read(f,as_version=4)
        ep    = ExecutePreprocessor(timeout=None, kernel_name='sysml')
        n_out = flname
        try:
            print("Executing: {}".format(flname))
            ep.preprocess(nb, {'metadata': {'path': 'TestSuite/'}})
        except CellExecutionError:
            msg = 'Error executing the notebook "%s".\n' % f
            msg += 'See notebook "%s" for the traceback.' % n_out
            print(msg)
        except TimeoutError:
            msg = 'Timeout executing the notebook "%s".\n' % n
            print(msg)
        finally:
            # Write output file
            with open(n_out, mode='wt') as out_file: nbformat.write(nb, out_file)            
            subprocess.call("jupyter trust '{}'".format(n_out), shell=True)
            
print("Done.")

Executing: KernelMagic.ipynb
Executing: SysML - Annex B Example.ipynb
Executing: SysML - Internet Model.ipynb
Executing: SysML - State Charts.ipynb
Executing: SysML - Decision Example.ipynb
Executing: TestSuite/CircularImport.ipynb
Executing: TestSuite/VerificationTest.ipynb
Executing: TestSuite/BlockTest.ipynb
Executing: TestSuite/AliasTest.ipynb
Executing: TestSuite/QPETest.ipynb
Executing: TestSuite/PrivateImportTest.ipynb
Executing: TestSuite/RequirementTest.ipynb
Executing: TestSuite/ActionTest.ipynb
Executing: TestSuite/AnalysisTest.ipynb
Executing: TestSuite/ParameterTest.ipynb
Executing: TestSuite/ControlNodeTest.ipynb
Executing: TestSuite/ConjugationTest.ipynb
Executing: TestSuite/TradeStudyTest.ipynb
Executing: TestSuite/ConstraintTest.ipynb
Executing: TestSuite/StateTest.ipynb
Executing: TestSuite/DecisionTest.ipynb
Executing: TestSuite/VariabilityTest.ipynb
Executing: TestSuite/PartTest.ipynb
Executing: TestSuite/TextualRepresentationTest.ipynb
Executing: TestSuite/Connecti

## Running Locally

This is intended to be run locally. The notebooks are then retrieved and stored in the repository. To this, the following should be done:

```
make spin-up
```

Will start the Jupyter and API servers locally in a Docker container.

Start a section in the Jupyter server and run this notebook.

After it completes, you can then copy all the notebooks to your local drive by doing:

```
make get-notebooks
```

**NOTE**: Ensure that the docker container is still running when getting the notebooks - these are retrieved from the docker container running the Jupyter server.

And after that, they can all be checked in and pushed to github.